# **Activation Heatmaps**



So in this notebook, we will make a Class Activation Map (CAM) on an image. But first, What is a Class Activation Map?

take a look at the example:

![CAM Examples](https://raw.githubusercontent.com/anubhavmaity/Sports-Type-Classifier/master/readme_images/si_sports.png)

Class Activation Maps or **CAM**s are a great way to get the discriminative image regions used by a CNN (Convolutional Neural Network) to identify a specific class in an image. This is a useful technique for debugging and visualizing the contributing parts of the image to the model's prediction.

# **Getting started**

## **Import the libraries**

In [1]:
import cv2
import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import matplotlib.pyplot as plt
import matplotlib.image as mimg

Using TensorFlow backend.


## **The image we will put the activation heatmap on**

In [0]:
img_path = "data/cat.jpg"
img = mimg.imread(img_path)
plt.imshow(img)

## **Processing the image**

Now we will process the image by resizing it into a 224x224 image and then convert it into a numpy array

In [0]:
from keras.preprocessing import image

def process_image(input_img):
  img = image.load_img(input_img, target_size=(224, 224))
  np_arr = image.img_to_array(img)
  np_arr = np.expand_dims(np_arr, axis=0)
  np_arr = preprocess_input(np_arr)
  return np_arr

img = process_image(img_path)
img

In [0]:
img.shape

## **Using the VGG16 model**

Now we will be using a pre-trained model called VGG16. It refers to a deep convolutional network for object recognition developed and trained by Oxford's renown Visual Geometry Group (VGG).

In [0]:
model = VGG16(weights='imagenet')
model.summary()

## **Predicting**

In [0]:
preds = model.predict(img)
print('Predictions: ', decode_predictions(preds, top=5)[0])

In [0]:
np.argmax(preds[0])

In [0]:
predictions = pd.DataFrame(decode_predictions(preds)[0],columns=['col1','category','probability']).iloc[:,1:]
print('Prediction:',predictions.loc[0,'category'])

FYI: Tabby is a type of cat that is in the image, So the prediction went well.

## **Setting up the Grad-CAM**

In [0]:
from keras import backend as K

def make_grad_cam(inputs):
  np_argmax = np.argmax(preds[0])
  output = model.output[:, np_argmax]
  
  # Get the last convolutional layer in our model
  last_conv_layer = model.get_layer('block5_conv3')
  
  # Get the gradients of the outputs
  gradients = K.gradients(output, last_conv_layer.output)[0]
  pooled_grads = K.mean(gradients, axis=(0, 1, 2))

  iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
  pooled_grads_value, conv_layer_output_value = iterate([inputs])
  
  for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
  heatmap = np.mean(conv_layer_output_value, axis=-1)
  return heatmap

make_grad_cam(img)

## **Setting up the Heatmap**

In [0]:
def gen_heatmaps(image_input):
  heatmap = make_grad_cam(image_input)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  return heatmap

generated_heatmap = gen_heatmaps(img)
plt.matshow(generated_heatmap)

Now, we've generated our heatmap congrats! You can see by the shape of the heatmap in the hottest region of the image, it resembles the cat's head shape. Now it's time to superimpose our heatmap with the original image. We will be using the `cv2` module to do the superimposition process of our heatmap to the original image.

## **Superimposing Our Heatmap**

In [0]:
from google.colab import files

def superimpose_heatmap(image_input, hmap, intense_factor):
  img = cv2.imread(image_input)

  # Resizing the heatmap
  heatmap = cv2.resize(hmap, (img.shape[1], img.shape[0]))

  # Converting the heatmap values into rgb values
  heatmap = np.uint8(255 * heatmap)

  # Applying a colormap to the heatmap, which in this notebook we will use COLORMAP_RAINBOW
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_RAINBOW)

  superimposed_img = heatmap * intense_factor + img

  # Saving the superimposed image
  output = 'output/output.jpg'
  cv2.imwrite(output, superimposed_img)

  print("Finished generating image!")
  print("Output: " + output)

superimpose_heatmap(img_path, gen_heatmaps(img), 0.5)